# PITCH REPRESENTATION IN MUSX

An overview of pitches, key numbers, intervals, sets and matrices in musx.

<hr style="height:1px;color:gray">

Notebook imports:

In [ ]:
import sys
sys.path.append('/Users/taube/Software/musx')
from IPython.display import display, HTML
display(HTML("<style>.container { width:100% !important; }</style>"))
from random import randint
from musx import version, keynum, hertz, pitch, between, Pitch, Interval, PCSet, Matrix
print(f"musx version: {version}")

## Pitch representation

<img src="support/pitch-keynum-hertz.png" alt="pitch-keynum-hertz.png" width="90%"/>

musx provides mappings between various representations of musical pitch information:

- hertz - cycles per second  (e.g. 261.23, 440)
- midi key numbers - ordinal indexes of keys on a virtual keyboard  (e.g. 60, 72)
- floating point key numbers - *kkk.ccc* whose fraction is taken to be cents (60.25 is a quarter tone above C4)
- pitch names - strings containing letter, accidental & octave, ('C#4', "Bb1")
- Pitch objects - immutable name, key and hertz with methods. (Pitch("c4"))
- Interval objects - immutable object that encodes the distance between two Pitches.
- PCSet and Matrix objects - immutable pitch class sets and matrices.



## Pitch names

Pitch names are strings containing a pitch letter followed by an optional accidental and a required octave number.

<ul>
<li>pitch letters can be upper or lower case: 'C' 'D' 'E' 'F' 'G' 'A' 'B'</li>
<li>sharps are '#' or 's' and double sharps are '##' or 'ss' (s is short for sharp)</li>
<li>flats are 'b' or 'f' and  double flats are 'bb' or 'ff' (f is short for flat)</li>
<li>octaves are '00' '0' '1' '2' '3' '4' '5' '6' '7' '8' '9'
</ul>

Examples:  'Cs4', 'Bb2', 'ef3' 'Gbb2'

## Pitch mapping

This group of functions provides a mapping between three different pitch representations:

<ul>
<li>keynum(x)</li>
<li>pitch(x)</li>
<li>hertz(x)</li>
</ul>

Pitch name to key number:

In [ ]:
keynum("C#4")

Key number to hertz:

In [ ]:
hertz(61)

Pitch name to hertz:

In [ ]:
hertz("C#4")

Key number to Pitch object:

In [ ]:
pitch(61)

Use the optional hz parameter if the input is a hertz value instead of a key number:

In [ ]:
pitch(61, hz=True)

Lowest possible pitch:

In [ ]:
keynum("C00") 

Highest possible pitch:

In [ ]:
keynum('Abb9') 

's' is a python-safe symbol for 'sharp' (# is the comment character in python...) so can appear in variable names:

In [ ]:
fs = keynum('fs4') 
print(fs)

List of inputs produce lists of keynums:

In [ ]:
keynum(['C#3', "D2", "G#2"])

Embedded lists are recursively processed:

In [ ]:
keynum(["a4", ["d4", "e4"], ["f4", "a4", "c5"], "d5"])

Strings with spaces are processed as lists with octave numbers 'sticky':

In [ ]:
keynum('cs5 d e f3 g eb9')

Elements can be directly repeated using ',' :

In [ ]:
keynum('cs5,,, d,,, e,, f3, g') 

Lists and strings can be combined:

In [ ]:
keynum(['cs5 d e', "f3, d", 'g eb9']) 

Converting hertz values to keynums:

In [ ]:
keynum([220, 440, 880])

keynum supports string lists of hertz values:

In [ ]:
keynum("100 200 300 400 500")

Optional filt (filter) argument allows alternate formatting:

In [ ]:
keynum("100 200 300 400", filt=lambda x: round(x,2))

The pitch function maps note names, keynums and hertz values to Pitch objects:

In [ ]:
pitch("C#2")

Lists can be passed into the pitch function:

In [ ]:
pitch([k for k in range(60,72)])

By default pitch() assumes values are key numbers:

In [ ]:
pitch(53)

To convert a hertz value to a Pitch you must indicate the value is hertz:

In [ ]:
pitch(53, 'hz')

By default, accidental spelling chooses the accidental from the most simple key spelling, e.g. the key number66 will be spelled as F# (a keysig of 1 sharp), not Gb (a keysig of 5 flats):

In [ ]:
pitch(66)

You can specify the accidental used for a given key number or hertz value, using the values -2 to 2, where -2 is double-flat, -1 is flat, 0 is no sharps or flats, 1 is one sharp, and 2 is double-sharp:

In [ ]:
pitch(66, acc=-1)

In [ ]:
pitch(53, acc=1)

In [ ]:
pitch(53, acc=-2)

## The Pitch object

A Pitch is an invariant object representing equal tempered pitches. It can return information in hertz, keynum, pitch name, pitch class, and Pnum formats. Pitches can be compared using standard math relations and maintain proper spelling when complemented or transposed by an Interval.

If Pitch() is called with no arguments then an 'empty pitch' is returned.  An empty pitch can be used to denote musical rests or an 'empty' status distinguishable from all other pitches:

In [ ]:
p = Pitch()
print('p is empty:', p.is_empty())

Otherwise, Pitch() should be called with one argument, a string or a list of integer attributes:

In [ ]:
print(Pitch("Ab3"))

Creating a pitch from its three element attribute list [*letter* (0-6), *accidental* (0-4), *octave* (0-10)] is useful when computing pitches from code:

In [ ]:
attrs = [between(0, 7), between(0, 5),  between(3, 9)]
print(Pitch(attrs))

A pitch can return information in a variety of formats:

In [ ]:
a,b = Pitch("Ef5"), Pitch("E5")
print(f"a = {a}")
print(f"b = {b}")
print(f"b.hertz() = {b.hertz()}")
print(f"a.keynum() = {a.keynum()}")
print(f"b.pc() = {b.pc()}")
print(f"b.string() = {b.string()}")
print(f"a < b = {a < b}")

A Pitch is a named tuple so it is hygienic (immutable):

In [ ]:
bs3 = Pitch("B#3")
print(bs3)
print(isinstance(bs3, tuple))
a,b,c = bs3
print(a,b,c)
print(list(bs3))
print(bs3._asdict())
try:
    bs3.letter = -99
except:
    print(f"Error: pitch letter {bs3.letter} cannot be altered.")

Pitch strings allow upper or lower case pitch letters ('C' or 'c'), and 'symbolic' or 'safe' versions of accidentals. 

The symbolic versions are ['bb', 'b', '', '#', ##']. 

The safe versions are ['ff', 'f', 'n', 's', 'ss']. 

The safe versions can be used in variable names or if you are feeling too lazy to use the shift key when you type ;) Pitches always display symbolic accidentals:

In [ ]:
print([Pitch("af4"), Pitch("gs4"), Pitch("Fff4")])

Pitch's `__repr__()` method  produces a string that, if evaluated, will re-creates the pitch: 

In [ ]:
repr(Pitch('F##00'))

Pitch's `__str__()` method should display the class name, the pitch string and the object's id inside <>, similar to Ratio:

In [ ]:
print(Pitch([3, 4, 0]))

The `string()` method returns just the pitch string name:

In [ ]:
p = Pitch('fss00')
print(f"p.string() = {p.string()}")
p

A Pitch has three attributes: letter, accidental and octave. Execute this next cell several times to see the tuple values:

In [ ]:
a = Pitch.random()
print('pitch:', repr(a))
print('tuple:', tuple(a))
print('a.letter:', a.letter)
print('a.accidental:', a.accidental)
print('a.octave:', a.octave)

Pitches can be compared using the arithmetic relations ==, !=, <, <=, >=, and > :

In [ ]:
l = [Pitch.random() for _ in range(8)]
for p1,p2 in zip(l, l[1:]):
    print(p1.string(),"<=", p2.string(), " -> ", p1 <= p2 )

How to sort a list of random pitches:

In [ ]:
l = [Pitch.random() for i in range(5)]
print("unsorted:", l)
print("sorted:  ", sorted(l))

You can convert a Pitch into a MIDI keynum:

In [ ]:
for p in [Pitch.random() for i in range(5)]:
    print(p.string(), "=>", p.keynum())

You can convert a Pitch into a hertz value:

In [ ]:
for p in [Pitch.random() for i in range(5)]:
    print(p.string(), "=>", p.hertz())

You can convert a Pitch into pitch classes:

In [ ]:
for p in [Pitch.random() for i in range(5)]:
    print(p.string(), "=>", p.pc())

Pitches can also return 'Pnums' (pitch nums).  A Pnum is an 
[IntEnum](https://docs.python.org/3/library/enum.html#enum.IntEnum) that 
enumerates all the pitch letters and accidentals within an octave.
Pnums are a bit like pitch classes (PCs) in the sense that they represent
pitches without respect to octaves.  Like PCs and keynums, Pnums can be
compared and sorted. But -- unlike PCs and keynums -- the Pnum's integer value
encodes and preserves the letter and accidental information of a pitch:

In [ ]:
print("Css < Dff:", Pitch.pnums.Css < Pitch.pnums.Dff)

# a pnum integer encodes its letter an accidental values
print("\nCss letter:", (Pitch.pnums.Css.value & 0xF0)>>4,
      ", accidental:", Pitch.pnums.Css.value & 0x0F, '\n')

l = [Pitch.random() for i in range(5)]

for p in l:
    print(p.pnum(), "name:", p.pnum().name , "value:", p.pnum().value)
    
sorted( [p.pnum() for p in l] )

## The Interval object

A musical interval measures the distance between two Pitches. This distance
can be measured in variety ways, for example lines-and-spaces, semitones, ratios, and cents. In classical music theory an interval distance is
measured using the number of spanning lines and spaces (unison, second, third, etc) together with a (possible) chromatic adjustment called a 'quality', e.g. diminished, augmented, major, perfect. 

The Interval object supports the classical interval system, including the notion of descending or ascending intervals and simple or compound intervals. Intervals can be numerically compared for their size (span plus quality) and can be used to transpose Pitches.


You can call the Interval constructor with one or two arguments, a single argument must be an interval string or an interval list: 

In [ ]:
i=Interval('P5')
print(f"i = {i}")
print(f"repr(i) = {repr(i)}")
j=Interval([2,5,0,1])
print(f"j = {j}")
print(f"repr(j) = {repr(j)}")
print(f'j.transpose(Pitch("Cs4")) {j.transpose(Pitch("Cs4"))}')

Intervals less than or equal to an 8va are said to be *simple* intervals, otherwise they are *compound* intervals greater than an octave:

In [ ]:
print(Interval("m2").is_simple())
print(Interval("P8").is_simple())
print(Interval("+8").is_simple())
print(Interval("o9").is_simple())
print(Interval("o9").is_compound())

If two arguments are passed, they must both be pitches, in this case the interval will be determined from the distance between the two pitches:

In [ ]:
a=Pitch('Ef3')
b=Pitch('D4')
print(a,b)
Interval(a, b)

If the first pitch is above the second pitch a 'descending' interval is formed:

In [ ]:
Interval(Pitch('Ef4'), Pitch("D3"))

It is also possible to specify a descending interval by preceding the interval name with a minus sign:

In [ ]:
print(Interval('-P5'))

The minus sign does not mean negative it means descending! The direction is particularly useful for analyzing melodic motion:

In [ ]:
m = ['e','e','f','g','g','f','e','d','c','c','d','e','e','d','d']
l = [Interval(Pitch(a+'4'), Pitch(b+'4')) for a,b in zip(m[:],m[1:])]

for i in l:
    print(i.string())

An Interval holds four integer attributes:

* `span`  - the number of lines and spaces spanned by the interval (0-7)
* `qual` -  an interval quality (0-12)
* `xoct` - the 'extra' octaves in a compound interval (0-10)
* `sign` - the direction,  -1 for descending, 1 for ascending

In [ ]:
i = Interval("-M9")
print("-M9: span =", i.span, ", qual =", i.qual, ", xoct =", i.xoct, ", sign =", i.sign)
i = Interval("+2")
print("+2: span =", i.span, ", qual =", i.qual, ", xoct =", i.xoct, ", sign =", i.sign)
i = Interval("P5")
print("P5: span =", i.span, ", qual =", i.qual, ", xoct =", i.xoct, ", sign =", i.sign)

An interval's `span` attribute ranges 0-7 inclusive. Span can be mapped to 
lines and spaces 1-8, names ['unison', 'second' ... 'octave'],
and full names.

In [ ]:
i=Interval('P5')
print('span =', i.span,
      ', lines and spaces =', i.lines_and_spaces(),
      ', name =', i.span_name(),
      ', full name =', i.full_name(), '\n')
i=Interval('-M2')
print('span =', i.span,
      ', lines and spaces =', i.lines_and_spaces(),
      ', name =', i.span_name(),
      ', full name =', i.full_name())

The Interval's `qual` attribute ranges 0-12 and can be mapped to symbolic or 'safe' quality values ranging from quintuply-diminished to quintuply-augmented: 'ooooo' ... 'o','m','P',"M",'+' ... '+++++' :

In [ ]:
print(Interval(Pitch('Bbb4'), Pitch('F##5')))
print(Interval(Pitch('Bbb4'), Pitch('F#5')))
print(Interval(Pitch('Bb4'), Pitch('F#5')))
print(Interval(Pitch('Bb4'), Pitch('F5')))
print(Interval(Pitch('B4'), Pitch('F5')))
print(Interval(Pitch('B4'), Pitch('Fb5')))
print(Interval(Pitch('B#4'), Pitch('Fb5')))
print(Interval(Pitch('B#4'), Pitch('Fbb5')))
print(Interval(Pitch('B##4'), Pitch('Fbb5')))

An interval's `xoct` (extra octaves) attribute is 0 for simple intervals and a positive integer for compound intervals:

In [ ]:
i=Interval('M2')
print('span=', i.span, 'xoct=', i.xoct)
i=Interval('M9')
print('span=', i.span, 'xoct=', i.xoct)
i=Interval('M16')
print('span=', i.span, 'xoct=', i.xoct)
i=Interval('M23')
print('span=', i.span, 'xoct=', i.xoct)

An interval's `sign` attribute is 1 for ascending intervals and -1 for
descending intervals:

In [ ]:
i=Interval('M6')
print('sign=', i.sign)
i=Interval('-M6')
print('sign=', i.sign)

Intervals can be reduced to semitones:

In [ ]:
print(Interval('ooooo5').semitones())
print(Interval('M2').semitones())

Intervals can be complemented:

In [ ]:
print(Interval("+6").complemented())
print(Interval("M2").complemented())
print(Interval("P5").complemented())

Intervals can be compared. Their order (position in 'interval space') depends on their spelling.  The `span` index has the most 'weight', so any smaller span will be less than any larger, even if the smaller span has more semitones. Within a single span the `qual` index determines the ordering:

In [ ]:
print(f"+5 has {Interval('+5').semitones()} semitones") 
print(f"o6 has {Interval('o6').semitones()} semitones")
print(f"+5 is smaller than o6: {Interval('+5') < Interval('o6')}")

Within the same span, interval quality is compared:

In [ ]:
Interval("oo5") < Interval("o5") < Interval("P5") < Interval("+5")

Since Intervals are ordered they can be sorted:

In [ ]:
l=[Interval("+4"), Interval("P5"), Interval("m3"), Interval("o3")]
print(f"unsorted intervals: {l}")
print(f"same list sorted:   {sorted(l)}")

Intervals can be inspected using a number of different methods:

In [ ]:
i=Interval('P5')
print(i.is_perfect())
print(i.is_fifth())
print(i.is_consonant())
print(i.is_simple())

i=Interval('m9')
print(i.is_minor())
print(i.is_second())
print(i.is_dissonant())
print(i.is_compound())

`is_perfect_type()` and `is_imperfect_type()` are more general predicates, they return true if the interval's span is a member of the perfect spans (1,4,5,8) or imperfect spans (2 3 6 7).

In [ ]:
print([Interval(s).is_perfect_type() for s in ['d5','d4', '+8', 'm2']])

print([Interval(s).is_imperfect_type() for s in ['m2','+3','d7', 'd5']])

The methods `is_diminished(`) and `is_augmented()` return true or false but the 'true' value will be the actual degree of diminution or augmentation:

In [ ]:
print(Interval('o3').is_augmented())
print(Interval('o3').is_diminished())
print(Interval('oo3').is_diminished())
print(Interval('ooo3').is_diminished())

Intervals can be added:

In [ ]:
print("P5 + m2:", Interval('P5').add(Interval('m2')))

print("P5 + +1:", Interval('P5').add(Interval('+1')))

Intervals can be used to transpose Pitches with proper spelling preserved:

In [ ]:
print(Interval('M3').transpose(Pitch('D#4')))

print(Interval('-M3').transpose(Pitch('D#4')))

In [ ]:
p=Pitch("C4")

l=[Interval(n) for n in ["P5", "+6", "m6", "+4", "P4", "o4", "P1", "M6", "+6","m7"]]

for i in l:
    print(p.string(),"+", i.string(), "==", i.transpose(p))

`transpose()` also works with pnums. pnums have no octave so they should be thought of as always ascending:

In [ ]:
print(Interval('m3').transpose(Pitch.pnums.D))
print(Interval('m3').transpose(Pitch.pnums.Ds))
print(Interval('M3').transpose(Pitch.pnums.Ds))
print(Interval('-m6').transpose(Pitch.pnums.Ds))

## Pitch classes, PCSet, and Matrix

A pitch class is just an integer 0-11 inclusive:

In [ ]:
Pitch("C#5").pc()

PCSet is an invariant class for working with pitch sets. To create a PCSet pass a list of Pitches, keynums, or pcs:

In [ ]:
print( PCSet([Pitch("Eb5"), Pitch("C4"), Pitch("F2")]) )  # pitches
print( PCSet([70, 70, 41, 32, 30, 30, 30]))  # keynums
print( PCSet([11,4,2]) ) # pcs

The PCSet's tuple data is stored in its 'set' attribute:

In [ ]:
s = PCSet([11,4,2])
print(s)
print(s.set)

Given a PCSet you can access its normal and prime forms, which are returned as PCSets:

In [ ]:
s = PCSet([Pitch("Eb5"), Pitch("C4"), Pitch("F2")]) 
print("set:", s)
print("normal form:", s.normalform())
print("prime form:", s.primeform())

The PCset's interval vector is returned as a list, not a tuple:

In [ ]:
print("interval vector:", s.intervalvector())

PCSet has `transpose()` and `invert()` methods:

In [ ]:
s = PCSet(keynum('g4 a bf'))
print(s)
print("transpose by tritone:", s.transpose(6))
print("invert at T0", s.invert())
print("invert at T5", s.invert(5))

You can create a matrix from a PCSet using its `matrix()` method. A matrix stores its data in its 'matrix' attribute, which stores the information as a 'row major' tuple of tuples starting on pitch class 0:

In [ ]:
berg = PCSet([k for k in keynum('g3 bf d4 fs a c5 e af b cs6 ds f')])
print(berg)
bergmatrix = berg.matrix()
print(bergmatrix)
print("matrix rows:", bergmatrix.matrix)

Given the matrix you can access its row forms and transpositions using labels like 'p0', 'ri3', 'i7' 'r3':

In [ ]:
print(bergmatrix.row('p7'))

for l in ['p11', 'ri3', 'i11', 'r3']:
    print(l + ':', bergmatrix.row(l).set)

The `matrix.print()` method will "pretty print" the matrix displaying either pitch classes or pitch names:

In [ ]:
bergmatrix.print()

In [ ]:
bergmatrix.print(True)